In [2]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
import pickle
import os
import keras_bert
import zipfile
from sklearn.datasets import load_files
import chardet
from keras.preprocessing.sequence import pad_sequences
import torch.nn.functional as func

/home/tanisha17116/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tanisha17116/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tanisha17116/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tanisha17116/.local/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
def standardize_to_utf8(encoding):
    """
    standardize to utf-8 if necessary.
    NOTE: mainly used to use utf-8 if ASCII is detected, as
    BERT performance suffers otherwise.
    """
    encoding = 'utf-8' if encoding.lower() in ['ascii', 'utf8', 'utf-8'] else encoding
    return encoding

In [3]:
train_test_names = ["train", "test"]
datadir = "dataset_up"
classes = ["class0", "class1"]
train_str = train_test_names[0]
train_b = load_files(os.path.join(datadir, train_str), shuffle=False, categories=classes)
test_str = train_test_names[1]
test_b = load_files(os.path.join(datadir,  test_str), shuffle=False, categories=classes)
x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

In [4]:
lst = [chardet.detect(doc)['encoding'] for doc in x_train[:32]]
encoding = max(set(lst), key=lst.count)
encoding = standardize_to_utf8(encoding)
x_train = [x.decode(encoding) for x in x_train]
x_test = [x.decode(encoding) for x in x_test]

In [5]:
len(x_train)

25788

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [7]:
def get_token_ids(x_train, x_test):
    
    token_tr = []
    token_tst = []
    count = 0
    for sent in x_train :
        tokens = tokenizer.encode(sent, add_special_tokens = True, max_length=512)
        token_tr.append(tokens)
        count+=1
        if(count%1000==0):
            print(count)
    
    for sent1 in x_test :
        tokens1 = tokenizer.encode(sent1, add_special_tokens = True, max_length=512)
        token_tst.append(tokens1)
        count+=1
        if(count%1000==0):
            print(count)
            
    return token_tr, token_tst 

In [8]:
xtr_token, xtst_token = get_token_ids(x_train, x_test)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000


In [9]:
xtr_token = pad_sequences(xtr_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")
xtst_token = pad_sequences(xtst_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")

In [10]:
attention_mask_tr = []
attention_mask_tst = []
for sent in xtr_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tr.append(att_mask)

In [11]:
for sent in xtst_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tst.append(att_mask)

In [26]:
# pickle.dump(xtr_token, open("pickles/train_input_tokens.pkl", "wb"))
# pickle.dump(xtst_token, open("pickles/test_input_tokens.pkl", "wb"))
# pickle.dump(attention_mask_tr, open("pickles/attention_mask_train.pkl" , "wb"))
# pickle.dump(attention_mask_tst, open("pickles/attention_mask_test.pkl", "wb"))

In [12]:
train_input = torch.tensor(xtr_token)
test_input = torch.tensor(xtst_token)

train_label = torch.tensor(y_train)
test_label = torch.tensor(y_test)

train_mask = torch.tensor(attention_mask_tr)
test_mask = torch.tensor(attention_mask_tst)

In [13]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32

Train_data = TensorDataset(train_input, train_mask, train_label)
Train_sampler = SequentialSampler(Train_data)
Train_dataloader = DataLoader(Train_data, sampler=Train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
Test_data = TensorDataset(test_input, test_mask, test_label)
Test_sampler = SequentialSampler(Test_data)
Test_dataloader = DataLoader(Test_data, sampler=Test_sampler, batch_size=batch_size)

In [14]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_data = TensorDataset(train_input, train_mask, train_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
test_data = TensorDataset(test_input, test_mask, test_label)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [6]:
import torch.nn as nn
from transformers import BertModel

class ContextEmbeddings(nn.Module):

    def __init__(self, freeze_bert = True):
        super(ContextEmbeddings, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
                
        self.embedding_layer = nn.Linear(768, 512)
        torch.nn.init.xavier_uniform_(self.embedding_layer.weight)
        self.embedding_layer.bias.data.fill_(0.01)
        self.cls_layer = nn.Linear(512, 1)
        torch.nn.init.xavier_uniform_(self.cls_layer.weight)
        self.cls_layer.bias.data.fill_(0.01)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)

        #Obtaining the representation of [CLS] head
        embedding_rep = cont_reps[:, 0]
        embeds = self.embedding_layer(embedding_rep)
        embeds = func.relu(embeds)
#         cls_rep = embeds[:,0]
        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(embeds)

        return logits

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = ContextEmbeddings(freeze_bert = True).to(device)

In [13]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [14]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

In [50]:
def train(net, criterion, opti, train_loader, test_loader, max_eps):

    for ep in range(max_eps):
        
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()

            if (it + 1) % 1 == 0:
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss : {} Accuracy : {}".format(it+1, ep+1, loss.item(), acc))

In [ ]:
train(net, criterion, opti, train_dataloader, test_dataloader, 4)

Iteration 1 of epoch 1 complete. Loss : 0.5709898471832275 Accuracy : 0.875
Iteration 2 of epoch 1 complete. Loss : 0.5330638885498047 Accuracy : 0.9375
Iteration 3 of epoch 1 complete. Loss : 0.5297664403915405 Accuracy : 0.9375
Iteration 4 of epoch 1 complete. Loss : 0.5648213624954224 Accuracy : 0.84375
Iteration 5 of epoch 1 complete. Loss : 0.4747810661792755 Accuracy : 0.96875
Iteration 6 of epoch 1 complete. Loss : 0.46808508038520813 Accuracy : 0.96875
Iteration 7 of epoch 1 complete. Loss : 0.513116180896759 Accuracy : 0.84375
Iteration 8 of epoch 1 complete. Loss : 0.4571857154369354 Accuracy : 0.9375
Iteration 9 of epoch 1 complete. Loss : 0.4486050307750702 Accuracy : 0.9375
Iteration 10 of epoch 1 complete. Loss : 0.4029240906238556 Accuracy : 0.96875
Iteration 11 of epoch 1 complete. Loss : 0.42757290601730347 Accuracy : 0.9375
Iteration 12 of epoch 1 complete. Loss : 0.4493497610092163 Accuracy : 0.90625
Iteration 13 of epoch 1 complete. Loss : 0.40939217805862427 Accura

Iteration 107 of epoch 1 complete. Loss : 0.3242058753967285 Accuracy : 0.90625
Iteration 108 of epoch 1 complete. Loss : 0.31794828176498413 Accuracy : 0.90625
Iteration 109 of epoch 1 complete. Loss : 0.25826749205589294 Accuracy : 0.9375
Iteration 110 of epoch 1 complete. Loss : 0.461330771446228 Accuracy : 0.84375
Iteration 111 of epoch 1 complete. Loss : 0.524827241897583 Accuracy : 0.8125
Iteration 112 of epoch 1 complete. Loss : 0.4413797855377197 Accuracy : 0.84375
Iteration 113 of epoch 1 complete. Loss : 0.2994282841682434 Accuracy : 0.90625
Iteration 114 of epoch 1 complete. Loss : 0.2582840323448181 Accuracy : 0.9375
Iteration 115 of epoch 1 complete. Loss : 0.23391863703727722 Accuracy : 0.9375
Iteration 116 of epoch 1 complete. Loss : 0.24176079034805298 Accuracy : 0.9375
Iteration 117 of epoch 1 complete. Loss : 0.5358160138130188 Accuracy : 0.8125
Iteration 118 of epoch 1 complete. Loss : 0.4345898926258087 Accuracy : 0.84375
Iteration 119 of epoch 1 complete. Loss : 0.

Iteration 210 of epoch 1 complete. Loss : 0.3752800226211548 Accuracy : 0.875
Iteration 211 of epoch 1 complete. Loss : 0.4962935745716095 Accuracy : 0.8125
Iteration 212 of epoch 1 complete. Loss : 0.246051624417305 Accuracy : 0.9375
Iteration 213 of epoch 1 complete. Loss : 0.2294645607471466 Accuracy : 0.9375
Iteration 214 of epoch 1 complete. Loss : 0.3727647066116333 Accuracy : 0.875
Iteration 215 of epoch 1 complete. Loss : 0.3658784329891205 Accuracy : 0.875
Iteration 216 of epoch 1 complete. Loss : 0.18039849400520325 Accuracy : 0.96875
Iteration 217 of epoch 1 complete. Loss : 0.16866834461688995 Accuracy : 0.96875
Iteration 218 of epoch 1 complete. Loss : 0.09776192903518677 Accuracy : 1.0
Iteration 219 of epoch 1 complete. Loss : 0.45689594745635986 Accuracy : 0.84375
Iteration 220 of epoch 1 complete. Loss : 0.24304978549480438 Accuracy : 0.9375
Iteration 221 of epoch 1 complete. Loss : 0.3737296164035797 Accuracy : 0.875
Iteration 222 of epoch 1 complete. Loss : 0.55911594

Iteration 314 of epoch 1 complete. Loss : 0.5352298617362976 Accuracy : 0.78125
Iteration 315 of epoch 1 complete. Loss : 0.24874338507652283 Accuracy : 0.9375
Iteration 316 of epoch 1 complete. Loss : 0.22733451426029205 Accuracy : 0.9375
Iteration 317 of epoch 1 complete. Loss : 0.3787602484226227 Accuracy : 0.875
Iteration 318 of epoch 1 complete. Loss : 0.10276566445827484 Accuracy : 1.0
Iteration 319 of epoch 1 complete. Loss : 0.15545925498008728 Accuracy : 0.96875
Iteration 320 of epoch 1 complete. Loss : 0.23911389708518982 Accuracy : 0.9375
Iteration 321 of epoch 1 complete. Loss : 0.296213299036026 Accuracy : 0.90625
Iteration 322 of epoch 1 complete. Loss : 0.24457409977912903 Accuracy : 0.9375
Iteration 323 of epoch 1 complete. Loss : 0.3091733157634735 Accuracy : 0.875
Iteration 324 of epoch 1 complete. Loss : 0.24511009454727173 Accuracy : 0.90625
Iteration 325 of epoch 1 complete. Loss : 0.35519343614578247 Accuracy : 0.875
Iteration 326 of epoch 1 complete. Loss : 0.246

Iteration 418 of epoch 1 complete. Loss : 0.2360352873802185 Accuracy : 0.9375
Iteration 419 of epoch 1 complete. Loss : 0.42025160789489746 Accuracy : 0.84375
Iteration 420 of epoch 1 complete. Loss : 0.3323921263217926 Accuracy : 0.875
Iteration 421 of epoch 1 complete. Loss : 0.2841224670410156 Accuracy : 0.90625
Iteration 422 of epoch 1 complete. Loss : 0.3339076042175293 Accuracy : 0.875
Iteration 423 of epoch 1 complete. Loss : 0.2074977308511734 Accuracy : 0.9375
Iteration 424 of epoch 1 complete. Loss : 0.2152089774608612 Accuracy : 0.9375
Iteration 425 of epoch 1 complete. Loss : 0.09013529121875763 Accuracy : 1.0
Iteration 426 of epoch 1 complete. Loss : 0.24770162999629974 Accuracy : 0.9375
Iteration 427 of epoch 1 complete. Loss : 0.29565683007240295 Accuracy : 0.90625
Iteration 428 of epoch 1 complete. Loss : 0.41615164279937744 Accuracy : 0.84375
Iteration 429 of epoch 1 complete. Loss : 0.2902584671974182 Accuracy : 0.90625
Iteration 430 of epoch 1 complete. Loss : 0.507

Iteration 521 of epoch 1 complete. Loss : 0.14534655213356018 Accuracy : 0.96875
Iteration 522 of epoch 1 complete. Loss : 0.23012186586856842 Accuracy : 0.9375
Iteration 523 of epoch 1 complete. Loss : 0.2677767872810364 Accuracy : 0.90625
Iteration 524 of epoch 1 complete. Loss : 0.40832266211509705 Accuracy : 0.875
Iteration 525 of epoch 1 complete. Loss : 0.35578450560569763 Accuracy : 0.875
Iteration 526 of epoch 1 complete. Loss : 0.26295405626296997 Accuracy : 0.90625
Iteration 527 of epoch 1 complete. Loss : 0.16405972838401794 Accuracy : 0.96875
Iteration 528 of epoch 1 complete. Loss : 0.5062030553817749 Accuracy : 0.78125
Iteration 529 of epoch 1 complete. Loss : 0.20678922533988953 Accuracy : 0.9375
Iteration 530 of epoch 1 complete. Loss : 0.2982042133808136 Accuracy : 0.90625
Iteration 531 of epoch 1 complete. Loss : 0.2509151101112366 Accuracy : 0.90625
Iteration 532 of epoch 1 complete. Loss : 0.28772279620170593 Accuracy : 0.90625
Iteration 533 of epoch 1 complete. Los

In [17]:
torch.save(net.state_dict(), "models/model_state_dict_512")

In [19]:
torch.save(net, "models/model_512")

/home/tanisha17116/.local/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ContextEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [15]:
model = ContextEmbeddings(freeze_bert=  True)

In [16]:
model.load_state_dict(torch.load("models/model_state_dict_512"))

<All keys matched successfully>

In [17]:
model.eval()

ContextEmbeddings(
  (bert_layer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
output_train = []
for it, (seq, attn_masks, labels) in enumerate(Train_dataloader):
    bert_out, _ = model.bert_layer.forward(seq, attn_masks)
    embeds = model.embedding_layer.forward(bert_out[:,0])
    output_train.append(embeds)
    print(it)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
pickle.dump(output_train, open("models/output_train_512.pkl","wb"))
print("Output_train_pickled...YAYYYYY")

In [ ]:
output_test = []
for it, (seq, attn_masks, labels) in enumerate(Test_dataloader) :
    bert_out, _ = model.bert_layer.forward(seq, attn_masks)
    embeds = model.embedding_layer.forward(bert_out[:,0])
    output_test.append(embeds)
    print(it)

In [ ]:
pickle.dump(output_test, open("models/output_test_512.pkl","wb"))
print("Output_test_pickled...YAYYYYY")